In [1]:
import pandas as pd

# Scraping Wikipedia

We read from the HTML table using pandas directly. This returns a list of dataframes, and we are interested in the first dataframe. We take only the rows of the dataframe where `'Borough'` is _not_ `'Not assigned'`. We reset the index numbering.

In [63]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)[0]
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


At this point, the dataframe looks almost right, but the instructions imply that some postal codes may be listed twice. Let's check if any postal code appears more than once in the dataframe:

In [64]:
(pd.value_counts(df['Postal code']) >1).any()

False

Great, now we need to make sure the Neighborhoods are separated by commas instead of forward slashes.

In [65]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now we need to check for assigned boroughs with "Not assigned" neighborhoods.

In [71]:
(df['Neighborhood'] == 'Not assigned').any()

False

There do not appear to be any. Apparently the dataset has changed since the assignment was written.